First a utility script to download mnist and save it into a pickle context for later usage.

In [1]:
import numpy
from urllib import request
import gzip
import pickle
import os
import time
import sys
import hashlib
import random

filename = [
    ["training_images","train-images-idx3-ubyte.gz"],
    ["test_images","t10k-images-idx3-ubyte.gz"],
    ["training_labels","train-labels-idx1-ubyte.gz"],
    ["test_labels","t10k-labels-idx1-ubyte.gz"]
]

def download_mnist():
    base_url = "http://yann.lecun.com/exdb/mnist/"
    for name in filename:
        print("Downloading "+name[1]+"...")
        request.urlretrieve(base_url+name[1], name[1])
    print("Download complete.")

def save_mnist():
    mnist = {}
    for name in filename[:2]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = numpy.frombuffer(f.read(), numpy.uint8, offset=16).reshape(-1,28*28)
    for name in filename[-2:]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = numpy.frombuffer(f.read(), numpy.uint8, offset=8)
    with open("mnist.pkl", 'wb') as f:
        pickle.dump(mnist,f)
    print("Save complete.")

def init():
    
    if os.path.exists("./mnist.pkl"):
        print("Pkl detected. Skipping download.")
        return
    
    download_mnist()
    save_mnist()

def load():
    with open("mnist.pkl",'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

init()

Pkl detected. Skipping download.


Let's define some additional information. Input will be simple 60000 records with a feature size of 784 (28x28) pixels (values 0-255 (white-black)). Let's say there are 7 interested parties (Node A - G) who wish to contribute computation power. The analyst/researcher decides to evenly split the 784 pixels across 7 parties, therefore each party will have 112 pixels from each image in the dataset. Below we determine the allocation of pixels for each node. Node A is denoted 0 and Node G is denoted 6.

In [2]:
learning_rate = 10
seed = 42
epochs = 5000

X = numpy.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])

Y = numpy.array([
    [0],
    [1],
    [1],
    [0]
])

training_images_count = 4
testing_images_count = 4
training_features_count = 2
testing_features_count = 2

numpy.random.seed(seed)

train_X = X
train_Y = Y
test_X = X
test_Y = Y

scheduler_count = 2

w1 = numpy.random.randn(2, 3) * 0.01
b1 = numpy.zeros(3)
z1 = numpy.zeros(3)
a1 = numpy.zeros(3)
w2 = numpy.random.randn(3, 1) * 0.01
b2 = numpy.zeros(1)
z2 = numpy.zeros(1)
a2 = numpy.zeros(1)

In [3]:
def create_exposure_matrix(operation, a, b = None):
    
    # dot uniquely uses different shaped a, b matrices
    # - exposures stores which scheduler is allocated to which "input pixel" position
    # - enshuffles stores the index ordering of b to shuffled prior to being passed to the applicible scheduler (exposures)
    # - unshuffles stores the index ordering of b to unshuffle/normalize the result into an aggregatable format
    
    if operation == 'dot':
        
        exposures = numpy.random.choice(scheduler_count, a.shape[1])
        numpy.random.shuffle(exposures)

        enshuffles = numpy.zeros(b.shape)
        for row in range(b.shape[0]):
            enshuffles[row] = numpy.arange(b.shape[1])
            random.shuffle(enshuffles[row])

        unshuffles = numpy.zeros(b.shape)
        for row in range(b.shape[0]):
            for col in range(b.shape[1]):
                unshuffles[row][int(enshuffles[row][col])] = col
        
        return {
            'exposures': exposures,
            'enshuffles': enshuffles,
            'unshuffles': unshuffles
        }
    
    # all other operations use equal shaped a, b matrices
    # - exposures stores which scheduler is allocated to which position
    # - enshuffles stores the index ordering of b to shuffled prior to being passed to the applicible scheduler (exposures)
    # - unshuffles stores the index ordering of b to unshuffle/normalize the result into an aggregatable format
    
    exposures = numpy.random.choice(scheduler_count, a.shape[1])
    numpy.random.shuffle(exposures)

    enshuffles = numpy.zeros(a.shape)
    for row in range(a.shape[0]):
        enshuffles[row] = numpy.arange(a.shape[1])
        random.shuffle(enshuffles[row])

    unshuffles = numpy.zeros(a.shape)
    for row in range(a.shape[0]):
        for col in range(a.shape[1]):
            unshuffles[row][int(enshuffles[row][col])] = col
    
    return {
        'exposures': exposures,
        'enshuffles': enshuffles,
        'unshuffles': unshuffles
    }

In [4]:
previous_weights_hash = '!'

for epoch in range(epochs):
    
    # layer 1 forward
    z1 = numpy.dot(train_X, w1) + b1
#     print(create_exposure_matrix('dot', train_X, w1))
    a1 = 1 / (1 + numpy.exp(-z1))
#     print(create_exposure_matrix('sigmoid', z1))

    # layer 2 forward
    z2 = numpy.dot(a1, w2) + b2
#     print(create_exposure_matrix('dot', a1, w2))
    a2 = 1 / (1 + numpy.exp(-z2))
#     print(create_exposure_matrix('sigmoid', z2))
        
    # computing cost
    m = 2
    delta = a2 - train_Y
#     print(create_exposure_matrix('minus', a2, train_Y))
    delta_squared = numpy.square(delta)
#     print(create_exposure_matrix('square', delta))
        
    cost = 1 / (2 * m) * numpy.sum(delta_squared)
#     print(create_exposure_matrix('minus', a2, train_Y))
    cost = numpy.squeeze(cost)
    dY_hat = (-1 / m) * delta
        
    # layer 2 backward
    a2_dz = dY_hat * a2 * (1 - a2)
    z2_dw = numpy.dot(a2_dz.T, a1)
    z2_db = numpy.sum(a2_dz.T, axis = 1, keepdims = True)
    z2_da = numpy.dot(w2, a2_dz.T)
    
    # layer 1 backward
    a1_dz = z2_da.T * a1 * (1 - a1)
    z1_dw = numpy.dot(a1_dz.T, train_X)
    z1_db = numpy.sum(a1_dz.T, axis = 1, keepdims = True)
    z1_da = numpy.dot(w1, a1_dz.T)

    # computing training accuracy
    training_correct = 0
    for index in range(testing_images_count):
        predicted = numpy.argmax(a2[index])
        actual = test_Y[index]
        if predicted == actual:
            training_correct += 1
    training_accuracy = training_correct / testing_images_count
    
    # computing validation accuracy
    z1 = numpy.dot(train_X, w1)
    a1 = 1 / (1 + numpy.exp(-z1))
    z2 = numpy.dot(a1, w2)
    a2 = 1 / (1 + numpy.exp(-z2))
    validation_correct = 0
    for index in range(training_images_count):
        predicted = numpy.argmax(a2[index])
        actual = train_Y[index]
        if predicted == actual:
            validation_correct += 1
    validation_accuracy = validation_correct / training_images_count
    
    print(cost, training_accuracy, validation_accuracy)
    
    # save model
#     model_state = {
#         'epoch': epoch,
#         'seed': seed,
#         'w1': w1,
#         'w2': w2,
#         'cost': cost,
#         'training_accuracy': training_accuracy,
#         'validation_accuracy': validation_accuracy,
#         'dw1': z1_dw,
#         'dw2': z2_dw,
#         'prev': previous_weights_hash
#     }
#     weights_hash = hashlib.md5()
#     weights_hash.update(w1.tobytes())
#     weights_hash.update(w2.tobytes())
#     weights_hash = weights_hash.hexdigest()
#     model_file_path = './models/{}.pkl'.format(weights_hash)
#     with open(model_file_path, 'wb') as model_file:
#         pickle.dump(model_state, model_file)
#     previous_weights_hash = weights_hash
    
    print(z2_db.T[0])
    
    # update weights and bias
    w2 += (learning_rate * z2_dw.T)
    w1 += (learning_rate * z1_dw.T)
    b2 += (learning_rate * z2_db.T[0])
    b1 += (learning_rate * z1_db.T[0])

0.2500055455908725 0.5 0.5
[-0.00117743]
0.2500078621893994 0.5 0.5
[0.00140194]
0.2500111454922455 0.5 0.5
[-0.00166917]
0.2500157987991989 0.5 0.5
[0.00198726]
0.2500223910689304 0.5 0.5
[-0.00236575]
0.2500317290319479 0.5 0.5
[0.00281607]
0.25004494676498823 0.5 0.5
[-0.00335152]
0.25006364913737944 0.5 0.5
[0.00398801]
0.25009007761030255 0.5 0.5
[-0.00474375]
0.2501273891046089 0.5 0.5
[0.00564046]
0.2501799389011399 0.5 0.5
[-0.00670224]
0.2502537951960425 0.5 0.5
[0.00795739]
0.2503571305317587 0.5 0.5
[-0.00943545]
0.2505010627211458 0.5 0.5
[0.01116978]
0.25069984190340133 0.5 0.5
[-0.01319023]
0.250971840742965 0.5 0.5
[0.01552659]
0.25133811133712525 0.5 0.5
[-0.0181922]
0.2518224058817146 0.5 0.5
[0.02118923]
0.25244409371608506 0.5 0.5
[-0.02447723]
0.2532153995760741 0.5 0.5
[0.02798759]
0.25412356239155415 0.5 0.5
[-0.03157788]
0.25513145547308824 0.5 0.5
[0.0350819]
0.2561561694326899 0.5 0.5
[-0.03826448]
0.25710199159748437 0.5 0.5
[0.04093961]
0.25785200494927674 0.

[1.88781068e-05]
0.24999714859909408 0.5 0.5
[1.89045444e-05]
0.24999712855951428 0.5 0.5
[1.89315029e-05]
0.2499971083717113 0.5 0.5
[1.8958442e-05]
0.24999708803389295 0.5 0.5
[1.89858624e-05]
0.2499970675442419 0.5 0.5
[1.90133062e-05]
0.249997046900915 0.5 0.5
[1.90411984e-05]
0.24999702610204289 0.5 0.5
[1.9069151e-05]
0.2499970051457297 0.5 0.5
[1.90975246e-05]
0.2499969840300525 0.5 0.5
[1.91259906e-05]
0.24999696275306046 0.5 0.5
[1.91548549e-05]
0.24999694131277503 0.5 0.5
[1.91838395e-05]
0.2499969197071889 0.5 0.5
[1.92132037e-05]
0.2499968979342655 0.5 0.5
[1.92427126e-05]
0.24999687599193893 0.5 0.5
[1.9272586e-05]
0.24999685387811255 0.5 0.5
[1.93026253e-05]
0.2499968315906598 0.5 0.5
[1.93330169e-05]
0.249996809127422 0.5 0.5
[1.93635932e-05]
0.24999678648620915 0.5 0.5
[1.93945122e-05]
0.24999676366479853 0.5 0.5
[1.94256324e-05]
0.24999674066093447 0.5 0.5
[1.94570879e-05]
0.24999671747232743 0.5 0.5
[1.94887594e-05]
0.24999669409665387 0.5 0.5
[1.95207606e-05]
0.24999

0.24997447590771732 0.5 0.5
[4.09303889e-05]
0.24997404572507756 0.5 0.5
[4.12254668e-05]
0.24997360435562332 0.5 0.5
[4.15255118e-05]
0.24997315140010617 0.5 0.5
[4.18306448e-05]
0.24997268644113552 0.5 0.5
[4.21409905e-05]
0.24997220904217576 0.5 0.5
[4.24566776e-05]
0.24997171874648044 0.5 0.5
[4.27778389e-05]
0.2499712150759551 0.5 0.5
[4.31046115e-05]
0.2499706975299456 0.5 0.5
[4.34371372e-05]
0.24997016558394564 0.5 0.5
[4.37755623e-05]
0.24996961868821732 0.5 0.5
[4.41200378e-05]
0.249969056266318 0.5 0.5
[4.44707199e-05]
0.24996847771352615 0.5 0.5
[4.48277701e-05]
0.2499678823951587 0.5 0.5
[4.51913552e-05]
0.24996726964477012 0.5 0.5
[4.55616478e-05]
0.24996663876222572 0.5 0.5
[4.59388263e-05]
0.24996598901163697 0.5 0.5
[4.63230753e-05]
0.24996531961914897 0.5 0.5
[4.67145857e-05]
0.2499646297705675 0.5 0.5
[4.71135552e-05]
0.24996391860881162 0.5 0.5
[4.75201883e-05]
0.2499631852311795 0.5 0.5
[4.79346969e-05]
0.2499624286864083 0.5 0.5
[4.83573003e-05]
0.2499616479715135

0.029348689473193676 0.5 0.5
[-0.00479243]
0.02784706664571209 0.5 0.5
[-0.00460055]
0.026453824319472095 0.5 0.5
[-0.00441824]
0.025160301140292343 0.5 0.5
[-0.00424525]
0.023958431707137375 0.5 0.5
[-0.00408125]
0.022840746540046572 0.5 0.5
[-0.00392587]
0.021800357945925546 0.5 0.5
[-0.00377873]
0.020830936259446146 0.5 0.5
[-0.0036394]
0.019926679928568947 0.5 0.5
[-0.00350748]
0.01908228206782637 0.5 0.5
[-0.00338256]
0.018292895418026048 0.5 0.5
[-0.00326426]
0.017554097105372438 0.5 0.5
[-0.00315217]
0.016861854170619243 0.5 0.5
[-0.00304593]
0.016212490515240194 0.5 0.5
[-0.00294519]
0.01560265566992474 0.5 0.5
[-0.00284962]
0.015029295612400809 0.5 0.5
[-0.00275891]
0.01448962573340241 0.5 0.5
[-0.00267274]
0.013981105959303769 0.5 0.5
[-0.00259085]
0.013501417978483431 0.5 0.5
[-0.00251297]
0.013048444478365801 0.5 0.5
[-0.00243887]
0.012620250275668968 0.5 0.5
[-0.0023683]
0.012215065209273132 0.5 0.5
[-0.00230106]
0.01183126866001509 0.5 0.5
[-0.00223694]
0.0114673755621291

0.0013984951492970001 0.5 0.5
[-0.00030329]
0.0013907247586253456 0.5 0.5
[-0.00030166]
0.001383035556675404 0.5 0.5
[-0.00030005]
0.0013754263124331823 0.5 0.5
[-0.00029846]
0.0013678958192068344 0.5 0.5
[-0.00029688]
0.0013604428940366033 0.5 0.5
[-0.00029532]
0.0013530663771217576 0.5 0.5
[-0.00029377]
0.0013457651312638547 0.5 0.5
[-0.00029224]
0.0013385380413258102 0.5 0.5
[-0.00029073]
0.0013313840137063192 0.5 0.5
[-0.00028923]
0.0013243019758290338 0.5 0.5
[-0.00028774]
0.0013172908756461097 0.5 0.5
[-0.00028627]
0.0013103496811555841 0.5 0.5
[-0.00028481]
0.0013034773799321965 0.5 0.5
[-0.00028337]
0.001296672978671164 0.5 0.5
[-0.00028194]
0.0012899355027445663 0.5 0.5
[-0.00028053]
0.0012832639957698825 0.5 0.5
[-0.00027912]
0.0012766575191903253 0.5 0.5
[-0.00027774]
0.0012701151518666029 0.5 0.5
[-0.00027636]
0.0012636359896796999 0.5 0.5
[-0.000275]
0.0012572191451444446 0.5 0.5
[-0.00027365]
0.0012508637470333881 0.5 0.5
[-0.00027231]
0.001244568940010779 0.5 0.5
[-0.000

...

6.863704772248387e-05 0.5 0.5
[-1.59115034e-05]
6.861410565360412e-05 0.5 0.5
[-1.5906266e-05]
6.859117862918288e-05 0.5 0.5
[-1.59010321e-05]
6.856826663454757e-05 0.5 0.5
[-1.58958015e-05]
6.854536965504867e-05 0.5 0.5
[-1.58905743e-05]
6.852248767605522e-05 0.5 0.5
[-1.58853505e-05]
6.849962068295465e-05 0.5 0.5
[-1.58801301e-05]
6.847676866115392e-05 0.5 0.5
[-1.58749131e-05]
6.845393159607733e-05 0.5 0.5
[-1.58696995e-05]
6.843110947316905e-05 0.5 0.5
[-1.58644893e-05]
6.840830227789296e-05 0.5 0.5
[-1.58592824e-05]
6.838550999572846e-05 0.5 0.5
[-1.58540789e-05]
6.836273261217852e-05 0.5 0.5
[-1.58488788e-05]
6.833997011276059e-05 0.5 0.5
[-1.58436821e-05]
6.83172224830134e-05 0.5 0.5
[-1.58384887e-05]
6.829448970849215e-05 0.5 0.5
[-1.58332986e-05]
6.827177177477341e-05 0.5 0.5
[-1.5828112e-05]
6.824906866744883e-05 0.5 0.5
[-1.58229287e-05]
6.822638037213137e-05 0.5 0.5
[-1.58177487e-05]
6.820370687445204e-05 0.5 0.5
[-1.58125721e-05]
6.818104816005968e-05 0.5 0.5
[-1.58073988e

[-1.4642229e-05]
6.306161233770714e-05 0.5 0.5
[-1.46377847e-05]
6.304218476977896e-05 0.5 0.5
[-1.46333432e-05]
6.302276894998423e-05 0.5 0.5
[-1.46289043e-05]
6.300336486775719e-05 0.5 0.5
[-1.4624468e-05]
6.298397251254268e-05 0.5 0.5
[-1.46200344e-05]
6.296459187380068e-05 0.5 0.5
[-1.46156035e-05]
6.294522294100344e-05 0.5 0.5
[-1.46111752e-05]
6.292586570363363e-05 0.5 0.5
[-1.46067496e-05]
6.290652015118758e-05 0.5 0.5
[-1.46023266e-05]
6.288718627317596e-05 0.5 0.5
[-1.45979063e-05]
6.286786405911932e-05 0.5 0.5
[-1.45934886e-05]
6.284855349855124e-05 0.5 0.5
[-1.45890736e-05]
6.282925458101925e-05 0.5 0.5
[-1.45846611e-05]
6.280996729608114e-05 0.5 0.5
[-1.45802514e-05]
6.279069163330929e-05 0.5 0.5
[-1.45758442e-05]
6.277142758228766e-05 0.5 0.5
[-1.45714397e-05]
6.275217513261095e-05 0.5 0.5
[-1.45670379e-05]
6.273293427388869e-05 0.5 0.5
[-1.45626386e-05]
6.271370499574171e-05 0.5 0.5
[-1.4558242e-05]
6.269448728780343e-05 0.5 0.5
[-1.4553848e-05]
6.267528113971796e-05 0.5 

5.9327747369987386e-05 0.5 0.5
[-1.37837037e-05]
5.931051398623107e-05 0.5 0.5
[-1.37797597e-05]
5.929329043214672e-05 0.5 0.5
[-1.37758179e-05]
5.927607669939317e-05 0.5 0.5
[-1.37718783e-05]
5.925887277963899e-05 0.5 0.5
[-1.3767941e-05]
5.924167866456439e-05 0.5 0.5
[-1.37640058e-05]
5.9224494345854014e-05 0.5 0.5
[-1.37600729e-05]
5.920731981520834e-05 0.5 0.5
[-1.37561423e-05]
5.919015506433074e-05 0.5 0.5
[-1.37522138e-05]
5.9173000084938266e-05 0.5 0.5
[-1.37482876e-05]
5.9155854868756064e-05 0.5 0.5
[-1.37443636e-05]
5.913871940751811e-05 0.5 0.5
[-1.37404418e-05]
5.9121593692968e-05 0.5 0.5
[-1.37365222e-05]
5.910447771685919e-05 0.5 0.5
[-1.37326048e-05]
5.9087371470952155e-05 0.5 0.5
[-1.37286896e-05]
5.907027494701878e-05 0.5 0.5
[-1.37247766e-05]
5.90531881368413e-05 0.5 0.5
[-1.37208658e-05]
5.9036111032206455e-05 0.5 0.5
[-1.37169573e-05]
5.901904362491648e-05 0.5 0.5
[-1.37130509e-05]
5.900198590677672e-05 0.5 0.5
[-1.37091467e-05]
5.898493786960735e-05 0.5 0.5
[-1.3705

5.636091044495828e-05 0.5 0.5
[-1.31044241e-05]
5.6345328953542374e-05 0.5 0.5
[-1.31008551e-05]
5.632975592321214e-05 0.5 0.5
[-1.30972879e-05]
5.631419134713152e-05 0.5 0.5
[-1.30937227e-05]
5.629863521847082e-05 0.5 0.5
[-1.30901594e-05]
5.628308753041094e-05 0.5 0.5
[-1.3086598e-05]
5.626754827613595e-05 0.5 0.5
[-1.30830385e-05]
5.625201744884036e-05 0.5 0.5
[-1.3079481e-05]
5.62364950417241e-05 0.5 0.5
[-1.30759253e-05]
5.622098104799639e-05 0.5 0.5
[-1.30723716e-05]
5.6205475460871264e-05 0.5 0.5
[-1.30688198e-05]
5.618997827357254e-05 0.5 0.5
[-1.30652699e-05]
5.617448947932869e-05 0.5 0.5
[-1.30617219e-05]
5.615900907137671e-05 0.5 0.5
[-1.30581757e-05]
5.614353704296105e-05 0.5 0.5
[-1.30546315e-05]
5.61280733873345e-05 0.5 0.5
[-1.30510892e-05]
5.611261809775571e-05 0.5 0.5
[-1.30475488e-05]
5.609717116748706e-05 0.5 0.5
[-1.30440103e-05]
5.608173258980572e-05 0.5 0.5
[-1.30404737e-05]
5.606630235798929e-05 0.5 0.5
[-1.3036939e-05]
5.605088046532628e-05 0.5 0.5
[-1.30334062e

[-1.21564498e-05]
5.221150743723829e-05 0.5 0.5
[-1.21533732e-05]
5.219809981382383e-05 0.5 0.5
[-1.21502982e-05]
5.218469895610501e-05 0.5 0.5
[-1.21472247e-05]
5.217130485900199e-05 0.5 0.5
[-1.21441527e-05]
5.215791751743838e-05 0.5 0.5
[-1.21410823e-05]
5.214453692634478e-05 0.5 0.5
[-1.21380134e-05]
5.213116308065451e-05 0.5 0.5
[-1.2134946e-05]
5.211779597530837e-05 0.5 0.5
[-1.21318802e-05]
5.210443560525058e-05 0.5 0.5
[-1.21288159e-05]
5.2091081965428964e-05 0.5 0.5
[-1.21257532e-05]
5.207773505080096e-05 0.5 0.5
[-1.21226919e-05]
5.206439485632461e-05 0.5 0.5
[-1.21196322e-05]
5.2051061376966064e-05 0.5 0.5
[-1.2116574e-05]
5.203773460769498e-05 0.5 0.5
[-1.21135174e-05]
5.202441454348354e-05 0.5 0.5
[-1.21104623e-05]
5.201110117931478e-05 0.5 0.5
[-1.21074087e-05]
5.199779451017069e-05 0.5 0.5
[-1.21043566e-05]
5.198449453104198e-05 0.5 0.5
[-1.2101306e-05]
5.1971201236924525e-05 0.5 0.5
[-1.2098257e-05]
5.19579146228166e-05 0.5 0.5
[-1.20952095e-05]
5.194463468372248e-05 0.

In [5]:
# define array for containing operations by node index
operations = [[] for x in range(scheduler_count)]

i_exposure_array = numpy.random.choice(scheduler_count, 784)
h_exposure_array = numpy.random.choice(scheduler_count, 128)
o_exposure_array = numpy.random.choice(scheduler_count, 10)

numpy.random.shuffle(i_exposure_array)
numpy.random.shuffle(h_exposure_array)
numpy.random.shuffle(o_exposure_array)

# divide operations for input_layer forward pass
for pixel_index in range(training_features_count):
    
    pixel_value = train_X[0][pixel_index]
    
    # skip adding computations where input is zero
    if pixel_value == 0:
        continue
    
    responsible_scheduler = i_exposure_array[pixel_index]    
    applicable_weights = w1[pixel_index]
    operations[responsible_scheduler].append([pixel_index, pixel_value, applicable_weights])

Next we define our activation function, its derivative, and the softmax function. 

In [6]:
def sigmoid(x):
    return 1 / (numpy.exp(-x) + 1)

def d_sigmoid(x):
    return numpy.exp(-x) / ((numpy.exp(-x) + 1) ** 2)

def softmax(x):
    exponents = numpy.exp(x)
    return exponents / numpy.sum(exponents)

def d_softmax(x):
    exp_element = np.exp(x - x.max())
    return exp_element / np.sum(exp_element, axis=0) * (1 - exp_element / np.sum(exp_element, axis=0))